In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import mysql.connector
from mysql.connector import errorcode
import pandas as pd
import numpy as np


[(1990, 'Driving Miss Daisy', 'O. Stone', "D. Day-Lewis",' J. Tandy' ),  ...]

In [ ]:
#[(1990, 'Driving Miss Daisy', 'O. Stone', "D. Day-Lewis",' J. Tandy' ),  ...]


url_oscar = "https://es.wikipedia.org/wiki/Premios_%C3%93scar"
res_oscar = requests.get(url_oscar)
sopa_oscar = BeautifulSoup(res_oscar.content, "html.parser") #traigo html
tabla_oscar = sopa_oscar.find("table", {"class": "wikitable"}) 
#en las tablas de wikipedia suelen tener "wikitable" como clase

# una vez tenemos la tabla:
# 1.sacamos todas las filas
# 2. una vez tenemos todas las filas, identifico la columna
# 3. Una vez tengo cada columnas, sacamos lo que nos interesa,nos piden el año solo(parseamos solo el año)
# una vez tenemos todos los valores, dejamos solo las filas y columnas que nos interesan:
#   fecha(año ceremonia),mejor peli, mejor director, mejor actor, mejor actriz
#   1990-2023

#como ya hemos traido html, sacamos una tabla de eso

def tabla_html_to_lista_tuplas(tabla_html): #el argumento es una tabla html
    lista_tuplas_oscars = []
    #print(tabla_html)
    filas = tabla_oscar.find_all("tr")
    for fila in filas:
        celdas = fila.find_all("td")
        lista_fila = []
        for celda in celdas:
            lista_fila.append(celda.text.strip())
        lista_tuplas_oscars.append(lista_fila)
    return lista_tuplas_oscars
    #nos la da sin limpiar

lista_tuplas_oscars = tabla_html_to_lista_tuplas(tabla_oscar)

#Ahora necesitamos que nos de la lista de tuplas "limpia"

def limpiar_lista_tuplas(lista_tuplas_oscars):
    lista_tuplas_limpia = []
    for fila in lista_tuplas_oscars:
        # quitamos la fila de cabecera que "está vacía", es decir, con etiqueta "th" no con "td" como las otras
        if fila != []:
            # llamamos a la función que hemos creado abajo para limpiar las filas
            fila_limpia = limpiar_fila(fila)
            lista_tuplas_limpia.append(fila_limpia)
    return lista_tuplas_limpia


# Ahora queremos limpiar cada fila. Le pasamos una fila sucia (tupla sucia) y 
# devuelve una fila limpia con el formato que nos piden:
# [(1990, 'Driving Miss Daisy', 'O. Stone', "D. Day-Lewis",' J. Tandy' ),  ...]

def limpiar_fila(fila_sucia):
    ceremonia = limpiar_celda_ceremonia(fila_sucia[1])
    pelicula = limpiar_celda_película(fila_sucia[2])
    director = limpiar_celda_nombres(fila_sucia[3])
    actor = limpiar_celda_nombres(fila_sucia[4])
    actriz = limpiar_celda_nombres (fila_sucia[5])
    fila_limpia = (ceremonia,pelicula,director,actor,actriz) # nos devuelve una tupla de todo lo anterior
    return fila_limpia 

def limpiar_celda_ceremonia(celda):
    celda_limpia = int(celda[-4:]) #convertimos a número el string que hemos separado y cogemos el 
                                             #ultimo string
    return celda_limpia

# la celda de Mejor Película ya me la da limpia la tabla original de html
# en caso de que haga falta cambiarla la dejamos hecha la función
def limpiar_celda_película(celda):
    return celda     

def limpiar_celda_nombres(celda):  # hay que limpiar el nombre de la pelicula
     celda_limpia = celda.split("(")[0] # que cuando se tope con "(" nos lo divida y nos quedamos con lo primero
     return celda_limpia


# Ahora hacemos la función de filtrar por año (1990-2023)
def filtrar_año(lista_tuplas_oscars,rango):
    lista_tuplas_oscars_filtrada = []
    for fila in lista_tuplas_oscars:
        if fila[0] in rango:
            lista_tuplas_oscars_filtrada.append(fila)

    return  lista_tuplas_oscars_filtrada

fila_limpia = limpiar_fila(lista_tuplas_oscars[80])


lista_tuplas_final = filtrar_año(limpiar_lista_tuplas(lista_tuplas_oscars),range(1990,2024))

lista_tuplas_final

   
    


In [13]:
df_oscars=pd.DataFrame(lista_tuplas_final)
df_oscars

,0,1,2,3,4
0,1990,Driving Miss Daisy,O. Stone,D. Day-Lewis,J. Tandy
1,1991,Dances with Wolves,K. Costner,J. Irons,K. Bates
2,1992,The Silence of the Lambs,J. Demme,A. Hopkins,J. Foster
3,1993,Unforgiven,C. Eastwood,A. Pacino,E. Thompson
4,1994,Schindler's List,S. Spielberg,T. Hanks,H. Hunter
5,1995,Forrest Gump,R. Zemeckis,T. Hanks,J. Lange
6,1996,Braveheart,M. Gibson,N. Cage,S. Sarandon
7,1997,The English Patient,A. Minghella,G. Rush,F. McDormand
8,1998,Titanic,J. Cameron,J. Nicholson,H. Hunt
9,1999,Shakespeare in Love,S. Spielberg,R. Benigni,G. Paltrow


In [5]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='bho_peliculas')

print(cnx)
cnx.close()

In [6]:
# En este código estamos haciendo un try except. Si recordamos esto nos permitía hacer un manejo de los errores, para evitar que nuestro código se pare. Para eso lo que estamos haciendo es
## intenta hacer la conexión son la base de datos de tienda 
try:
  cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='bho_peliculas')
# en caso de que no lo consigas por que hay algún error entonces ...
except mysql.connector.Error as err:

  # si es un error con la contraseña devuelveme un mensaje de acceso denegado ya que tenemos problemas con la contraseña
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  
  # si el error no tiene que ver con la contraseña, puede ser porque la base de datos no exista, devuelveme un mensaje de que la base de datos no existe
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  
  # si no es por ninguno de los errores anteriores, printeame cual es el error que estoy teniendo en mi conexión
  else:
    print(err)
else:
  cnx.close()

In [23]:
lista_tuplas_final

[(1990, 'Driving Miss Daisy', 'O. Stone', 'D. Day-Lewis', 'J. Tandy'),
 (1991, 'Dances with Wolves', 'K. Costner', 'J. Irons', 'K. Bates'),
 (1992, 'The Silence of the Lambs', 'J. Demme', 'A. Hopkins', 'J. Foster'),
 (1993, 'Unforgiven', 'C. Eastwood', 'A. Pacino', 'E. Thompson'),
 (1994, "Schindler's List", 'S. Spielberg', 'T. Hanks', 'H. Hunter'),
 (1995, 'Forrest Gump', 'R. Zemeckis', 'T. Hanks', 'J. Lange'),
 (1996, 'Braveheart', 'M. Gibson', 'N. Cage', 'S. Sarandon'),
 (1997, 'The English Patient', 'A. Minghella', 'G. Rush', 'F. McDormand'),
 (1998, 'Titanic', 'J. Cameron', 'J. Nicholson', 'H. Hunt'),
 (1999, 'Shakespeare in Love', 'S. Spielberg', 'R. Benigni', 'G. Paltrow'),
 (2000, 'American Beauty', 'S. Mendes', 'K. Spacey', 'H. Swank'),
 (2001, 'Gladiator', 'S. Soderbergh', 'R. Crowe', 'J. Roberts'),
 (2002, 'A Beautiful Mind', 'R. Howard', 'D. Washington', 'H. Berry'),
 (2003, 'Chicago', 'R. Polanski', 'A. Brody', 'N. Kidman'),
 (2004,
  'The Lord of the Rings: The Return of 

In [24]:
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='bho_peliculas')


mycursor = cnx.cursor()
sql = "INSERT INTO oscars (fecha_ceremonia, nombre_pelicula, direccion, nombre_actor, nombre_actriz) VALUES (%s, %s, %s, %s, %s)"
val = [(1990, 'Driving Miss Daisy', 'O. Stone', 'D. Day-Lewis', 'J. Tandy'),
 (1991, 'Dances with Wolves', 'K. Costner', 'J. Irons', 'K. Bates'),
 (1992, 'The Silence of the Lambs', 'J. Demme', 'A. Hopkins', 'J. Foster'),
 (1993, 'Unforgiven', 'C. Eastwood', 'A. Pacino', 'E. Thompson'),
 (1994, "Schindler's List", 'S. Spielberg', 'T. Hanks', 'H. Hunter'),
 (1995, 'Forrest Gump', 'R. Zemeckis', 'T. Hanks', 'J. Lange'),
 (1996, 'Braveheart', 'M. Gibson', 'N. Cage', 'S. Sarandon'),
 (1997, 'The English Patient', 'A. Minghella', 'G. Rush', 'F. McDormand'),
 (1998, 'Titanic', 'J. Cameron', 'J. Nicholson', 'H. Hunt'),
 (1999, 'Shakespeare in Love', 'S. Spielberg', 'R. Benigni', 'G. Paltrow'),
 (2000, 'American Beauty', 'S. Mendes', 'K. Spacey', 'H. Swank'),
 (2001, 'Gladiator', 'S. Soderbergh', 'R. Crowe', 'J. Roberts'),
 (2002, 'A Beautiful Mind', 'R. Howard', 'D. Washington', 'H. Berry'),
 (2003, 'Chicago', 'R. Polanski', 'A. Brody', 'N. Kidman'),
 (2004,
  'The Lord of the Rings: The Return of the King',
  'P. Jackson',
  'S. Penn',
  'C. Theron'),
 (2005, 'Million Dollar Baby', 'C. Eastwood', 'J. Foxx', 'H. Swank'),
 (2006, 'Crash', 'A. Lee', 'P. S. Hoffman', 'R. Witherspoon'),
 (2007, 'The Departed', 'M. Scorsese', 'F. Whitaker', 'H. Mirren'),
 (2008,
  'No Country for Old Men',
  'J. Coen E. Coen',
  'D. Day-Lewis',
  'M. Cotillard'),
 (2009, 'Slumdog Millionaire', 'D. Boyle', 'S. Penn', 'K. Winslet'),
 (2010, 'The Hurt Locker', 'K. Bigelow', 'J. Bridges', 'S. Bullock'),
 (2011, "The King's Speech", 'T. Hooper', 'C. Firth', 'N. Portman'),
 (2012, 'The Artist', 'M. Hazanavicius', 'J. Dujardin', 'M. Streep'),
 (2013, 'Argo', 'A. Lee', 'D. Day-Lewis', 'J. Lawrence'),
 (2014, '12 Years a Slave', 'A. Cuarón', 'M. McConaughey', 'C. Blanchett'),
 (2015,
  'Birdman or (The Unexpected Virtue of Ignorance)',
  'A. G. Iñárritu',
  'E. Redmayne',
  'J.Moore'),
 (2016, 'Spotlight', 'A. G. Iñárritu', 'L. DiCaprio', 'B. Larson'),
 (2017, 'Moonlight', 'D. Chazelle', 'C. Affleck', 'E. Stone'),
 (2018, 'The Shape of Water', 'G. del Toro', 'G. Oldman', 'F. McDormand'),
 (2019, 'Green Book', 'A. Cuarón', 'R. Malek', 'O. Colman'),
 (2020, 'Parasite', 'Bong J.', 'J. Phoenix', 'R. Zellweger'),
 (2021, 'Nomadland', 'C. Zhao', 'A. Hopkins', 'F. McDormand'),
 (2022, 'CODA', 'J. Campion', 'W. Smith', 'J. Chastain'),
 (2023,
  'Everything Everywhere All at Once',
  'D. Kwan D. Scheinert',
  'B. Fraser',
  'M. Yeoh')]

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`bho_peliculas`.`oscars`, CONSTRAINT `oscars_ibfk_1` FOREIGN KEY (`nombre_actor`) REFERENCES `datos_interpretes` (`nombre_interprete`) ON DELETE CASCADE ON UPDATE CASCADE)
Error Code: 1452
SQLSTATE 23000
Message Cannot add or update a child row: a foreign key constraint fails (`bho_peliculas`.`oscars`, CONSTRAINT `oscars_ibfk_1` FOREIGN KEY (`nombre_actor`) REFERENCES `datos_interpretes` (`nombre_interprete`) ON DELETE CASCADE ON UPDATE CASCADE)
